# dbnascent_initial_build

## This is a script for initially building dbnascent from metadata
### See dbnascent_update for updating existing database

### Load libraries

In [1]:
import os
import shutil
import numpy as np
import sqlalchemy as sql
import zipfile as zp
import re
#from . import utils #(in script)
#from . import orm #(in script)

### Import utilities (in notebook)

In [2]:
%load utils.py

### Import ORM (in notebook)

In [3]:
%load orm.py

### Load config file

In [4]:
#config = utils.load_config("/home/lsanford/Documents/data/repositories/dbnascent_build/config.txt")
config = load_config("/home/lsanford/Documents/data/repositories/dbnascent_build/config.txt")

### Create tables in database if they don't exist

In [5]:
db_url = config["file_locations"]["database"]

#utils.update_tables(db_url)
update_tables(db_url)

### Parse pre-created genome table and load into DB

In [6]:
organism_keys = list(dict(config["organism keys"]).values())

#organism_table = utils.get_unique_table("organism_table",organism_keys)
organism_table = get_unique_table("organism_table",organism_keys)

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(organism_table)):
        entry = organismInfo(
            organism = organism_table[i][config["organism keys"]["organism"]],
            genome_build = organism_table[i][config["organism keys"]["genome_build"]],
            genome_bases = organism_table[i][config["organism keys"]["genome_bases"]],
        )
        session.merge(entry)

### Parse metadata table and load exptMetadata values into DB, creating expt_id

In [7]:
expt_keys = list(dict(config["expt keys"]).values())

#expt_table = utils.get_unique_table("metadata",expt_keys)
expt_table = get_unique_table("metadata",expt_keys)

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(expt_table)):
        entry = exptMetadata(
            srp = expt_table[i][config["expt keys"]["srp"]],
            protocol = expt_table[i][config["expt keys"]["protocol"]],
            organism = expt_table[i][config["expt keys"]["organism"]],
            library = expt_table[i][config["expt keys"]["library"]],
            spikein = expt_table[i][config["expt keys"]["spikein"]],
            paper_id = expt_table[i][config["expt keys"]["paper_id"]],
            published = int(expt_table[i][config["expt keys"]["published"]]),
            year = expt_table[i][config["expt keys"]["year"]],
            first_author = expt_table[i][config["expt keys"]["first_author"]],
            last_author = expt_table[i][config["expt keys"]["last_author"]],
            doi = expt_table[i][config["expt keys"]["doi"]],
            curator1 = expt_table[i][config["expt keys"]["curator1"]],
            curator2 = expt_table[i][config["expt keys"]["curator2"]],
            other_sr_data = int(expt_table[i][config["expt keys"]["other_sr_data"]]),
            atac_seq = int(expt_table[i][config["expt keys"]["atac_seq"]]),
            rna_seq = int(expt_table[i][config["expt keys"]["rna_seq"]]),
            chip_seq = int(expt_table[i][config["expt keys"]["chip_seq"]]),
            three_dim_seq = int(expt_table[i][config["expt keys"]["three_dim_seq"]]),
            )
        session.merge(entry)

### Parse metadata table and load sampleID values into DB, creating sample_id

In [8]:
sample_keys = list(dict(config["sample keys"]).values())

#sample_table = utils.get_unique_table("metadata",sample_keys)
sample_table = get_unique_table("metadata",sample_keys)

#metatable = utils.table_parse(config["file_locations"]["metadata"])
#srz_list = np.array(utils.key_grab(metatable, [config["sample keys"]["sample_name"]]))
metatable = table_parse(config["file_locations"]["metadata"])
srz_list = np.array(key_grab(metatable, [config["sample keys"]["sample_name"]]))
srz_list = np.unique(srz_list[srz_list != ""])
srz_table = dict(zip(srz_list,list(range(1,len(srz_list)+1))))

z = len(srz_table) + 1
for i in range(len(sample_table)):
    if sample_table[i][config["sample keys"]["sample_name"]] == "":
        sample_table[i]["sample_id"] = z
        sample_table[i][config["sample keys"]["sample_name"]] = sample_table[i][config["sample keys"]["srr"]]
        z = z + 1
    else:
        sample_table[i]["sample_id"] = srz_table[sample_table[i][config["sample keys"]["sample_name"]]]
        
#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(sample_table)):
        entry = sampleID(
            sample_id = sample_table[i]["sample_id"],
            srr = sample_table[i][config["sample keys"]["srr"]],
            sample_name = sample_table[i][config["sample keys"]["sample_name"]],
            )
        session.merge(entry)

### Parse metadata table and load geneticInfo values into DB

In [9]:
genetic_keys = list(dict(config["genetic keys"]).values())

#genetic_table = utils.get_unique_table("metadata",genetic_keys)
genetic_table = get_unique_table("metadata",genetic_keys)

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(genetic_table)):
        genetic = geneticInfo(
            organism = genetic_table[i][config["genetic keys"]["organism"]],
            cell_type = genetic_table[i][config["genetic keys"]["cell_type"]],
            strain = genetic_table[i][config["genetic keys"]["strain"]],
            genotype = genetic_table[i][config["genetic keys"]["genotype"]],
            construct = genetic_table[i][config["genetic keys"]["construct"]],
            )
        session.merge(genetic)

### Parse metadata table and load conditionInfo values into DB

In [10]:
condition_keys = list(dict(config["condition keys"]).values())

#condition_table = utils.get_unique_table("conditions",condition_keys)
condition_table = get_unique_table("conditions",condition_keys)

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(condition_table)):
        if not condition_table[i]["end time"]:
            duration = ""
        else:
            duration = int(condition_table[i]["end time"]) - int(condition_table[i]["start time"])
            
        condition = conditionInfo(
            condition_type = condition_table[i][config["condition keys"]["condition_type"]],
            treatment = condition_table[i][config["condition keys"]["treatment"]],
            conc_intens = condition_table[i][config["condition keys"]["conc_intens"]],
            start_time = condition_table[i][config["condition keys"]["start_time"]],
            end_time = condition_table[i][config["condition keys"]["end_time"]],
            duration = duration,
            time_unit = condition_table[i][config["condition keys"]["time_unit"]],
            )
        session.merge(condition)

### Create exptCondition equivalencies in DB

In [11]:
# Pull condition info including ID from database and make a unique hash for conditions

condition_id = []
condition_details = []

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for row in session.query(conditionInfo).all():
        condition_id.append(row.condition_id)
        condition_details.append("".join([str(row.condition_type),
                                         str(row.treatment),
                                         str(row.conc_intens),
                                         str(row.start_time),
                                         str(row.end_time),
                                         str(row.time_unit)]))

condition_dict = dict(zip(condition_details,condition_id))

# Pull sample ID from database for each SRR

srr = []
sample_id = []

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for row in session.query(sampleID).all():
        srr.append(row.srr)
        sample_id.append(row.sample_id)
        
sample_dict = dict(zip(srr,sample_id))

# Grab condition table including SRRs and make SRR/condition hash

condition_keys = list(dict(config["condition keys"]).values())
condition_keys.append("srr")
#condition_table = utils.table_parse(config["file_locations"]["conditions"])
#cond_str = utils.key_grab(condition_table,condition_keys)
condition_table = table_parse(config["file_locations"]["conditions"])
cond_str = key_grab(condition_table,condition_keys)

srr_cond = []
for i in range(len(cond_str)):
    srr_cond.append([cond_str[i][-1],"".join(cond_str[i][0:-1])])

srr_cond = np.unique(np.array(srr_cond),axis=0)

# Make sample ID/condition ID table
sample_condition = []
for i in range(len(srr_cond)):
    sample_id = sample_dict[srr_cond[i][0]]
    condition = condition_dict[srr_cond[i][1]]
    sample_condition.append([sample_id,condition])

sample_condition = np.unique(np.array(sample_condition),axis=0)

# Add to database

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(sample_condition)):
        statement = exptCondition.insert().values(sample_id=int(sample_condition[i][0]), 
                                                  condition_id=int(sample_condition[i][1]))
        session.execute(statement)
        session.commit()
        

### Build sampleAccum table and linker table

#### Add relevant DB keys to table in correct rows

In [12]:
#metatable = utils.table_parse(config["file_locations"]["metadata"])
metatable = table_parse(config["file_locations"]["metadata"])

# Do some data massaging
for i in range(len(metatable)):
    metatable[i]["year"] = int(metatable[i]["year"])
    metatable[i]["replicate"] = metatable[i]["replicate"][0:4]
    if not metatable[i][config["sample keys"]["sample_name"]]:
        metatable[i][config["sample keys"]["sample_name"]] = metatable[i][config["sample keys"]["srr"]]
    for key in metatable[i]:
        if metatable[i][key] == '0':
            metatable[i][key] = False
        elif metatable[i][key] == '1':
            metatable[i][key] = True

# Add sample id
sample_keys = dict(config["sample keys"])
#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for row in session.query(sampleID).all():
        db_row = object_as_dict(row)
        for i in range(len(metatable)):
            if value_compare(db_row,metatable[i],sample_keys):
                metatable[i]["sample_id"] = row.sample_id
                metatable[i]["sample_name"] = row.sample_name

# Add genetic id
genetic_keys = dict(config["genetic keys"])
#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for row in session.query(geneticInfo).all():
        db_row = object_as_dict(row)
        for i in range(len(metatable)):
            if value_compare(db_row,metatable[i],genetic_keys):
                metatable[i]["genetic_id"] = row.genetic_id         

# Add experimental id                
expt_keys = dict(config["expt keys"])
#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for row in session.query(exptMetadata).all():
        db_row = object_as_dict(row)
        for i in range(len(metatable)):
            if value_compare(db_row,metatable[i],expt_keys):
                metatable[i]["expt_id"] = row.expt_id
                
# Collapse table to unique values based on sample_id (combines SRZs, essentially)
metatable = list({v['sample_id']:v for v in metatable}.values())

#### Make linkIDs table

In [13]:
#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(metatable)):
        entry = linkIDs(
            sample_id = metatable[i]["sample_id"],
            genetic_id = metatable[i]["genetic_id"],
            expt_id = metatable[i]["expt_id"],
        )
        session.merge(entry)

#### Load database data location

In [14]:
data_path = config["file_locations"]["data"]

#### Scrape FastQC data

In [15]:
for i in range(len(metatable)):
    paper_id = metatable[i][config["expt keys"]["paper_id"]]
    sample = metatable[i]["sample_name"]
    dirpath = data_path + paper_id + "/qc/fastqc/zips/"
    
    if metatable[i][config["accum keys"]["single_paired"]] == "paired":
        samp_zip = dirpath + sample + "_1_fastqc"
    else:
        samp_zip = dirpath + sample + "_fastqc"

    if not (
        os.path.exists(samp_zip + ".zip")
    ):
        continue
        
    with zp.ZipFile(samp_zip + ".zip", 'r') as zp_ref:
        zp_ref.extractall(dirpath)

    fdata = open(samp_zip + "/fastqc_data.txt")
    for line in fdata:
        if re.compile("Total Sequences").search(line):
            metatable[i]["raw_read_depth"] = int(line.split()[2])
        if re.compile("Sequence length").search(line):
            metatable[i]["raw_read_length"] = int(line.split()[2].split("-")[0])

    shutil.rmtree(samp_zip)

    if metatable[i][config["accum keys"]["rcomp"]] == 1:
        if metatable[i][config["accum keys"]["single_paired"]] == "paired":
            samp_zip = dirpath + sample + "_1.flip.trim_fastqc"
        else:
            samp_zip = dirpath + sample + ".flip.trim_fastqc"
    else:
        if metatable[i][config["accum keys"]["single_paired"]] == "paired":
            samp_zip = dirpath + sample + "_1.trim_fastqc"
        else:
            samp_zip = dirpath + sample + ".trim_fastqc"
    
    with zp.ZipFile(samp_zip + ".zip", 'r') as zp_ref:
        zp_ref.extractall(dirpath)

    fdata = open(samp_zip + "/fastqc_data.txt")
    for line in fdata:
        if re.compile("Total Sequences").search(line):
            metatable[i]["trim_read_depth"] = int(line.split()[2])

    shutil.rmtree(samp_zip)

#### Scrape picardtools data

In [16]:
for i in range(len(metatable)):
    paper_id = metatable[i][config["expt keys"]["paper_id"]]
    sample = metatable[i]["sample_name"]    
    dirpath = data_path + paper_id + "/qc/picard/dups/"
    filepath = dirpath + sample + ".marked_dup_metrics.txt"
    
    if not (
        os.path.exists(filepath) and os.path.isfile(filepath)
    ):
        continue
        
    fdata = open(filepath)
    for line in fdata:
        if re.compile("Unknown Library").search(line):
            metatable[i]["duplication_picard"] = float(line.split("\t")[8])

#### Scrape mapping data

In [17]:
for i in range(len(metatable)):
    paper_id = metatable[i][config["expt keys"]["paper_id"]]
    sample = metatable[i]["sample_name"]    
    dirpath = data_path + paper_id + "/qc/hisat2_mapstats/"
    filepath = dirpath + sample + ".hisat2_mapstats.txt"
    
    if not (
        os.path.exists(filepath) and os.path.isfile(filepath)
    ):
        continue
        
    fdata = open(filepath)
    if metatable[i][config["accum keys"]["single_paired"]] == "paired":
        for line in fdata:
            if re.compile("concordantly 1 time").search(line):
                reads = int(line.split(": ")[1].split(" (")[0]) * 2
            if re.compile("Aligned 1 time").search(line):    
                metatable[i]["single_map"] = reads + int(line.split(": ")[1].split(" (")[0])
            if re.compile("concordantly >1 times").search(line):
                reads = int(line.split(": ")[1].split(" (")[0]) * 2
            if re.compile("Aligned >1 times").search(line):
                metatable[i]["multi_map"] = reads + int(line.split(": ")[1].split(" (")[0])
            if re.compile("Overall alignment rate").search(line):
                metatable[i]["map_prop"] = float(line.split(": ")[1].split("%")[0])/100
    else:
        for line in fdata:
            if re.compile("Aligned 1 time").search(line):
                metatable[i]["single_map"] = int(line.split(": ")[1].split(" (")[0])
            if re.compile("Aligned >1 times").search(line):
                metatable[i]["multi_map"] = int(line.split(": ")[1].split(" (")[0])
            if re.compile("Overall alignment rate").search(line):
                metatable[i]["map_prop"] = float(line.split(": ")[1].split("%")[0])/100

#### Scrape rseqc data

In [18]:
for i in range(len(metatable)):
    paper_id = metatable[i][config["expt keys"]["paper_id"]]
    sample = metatable[i]["sample_name"] 
    dirpath = data_path + paper_id + "/qc/rseqc/read_distribution/"
    filepath = dirpath + sample + ".read_distribution.txt"
    
    if not (
        os.path.exists(filepath) and os.path.isfile(filepath)
    ):
        continue
        
    fdata = open(filepath)
    for line in fdata:
        if re.compile("Total Assigned Tags").search(line):
            metatable[i]["rseqc_tags"] = int(line.split()[-1])
        if re.compile("CDS_Exons").search(line):
            metatable[i]["rseqc_cds"] = int(line.split()[2])
            metatable[i]["cds_rpk"] = float(line.split()[-1])
        if re.compile("5'UTR_Exons").search(line):
            metatable[i]["rseqc_five_utr"] = int(line.split()[2])
        if re.compile("3'UTR_Exons").search(line):
            metatable[i]["rseqc_three_utr"] = int(line.split()[2])
        if re.compile("Introns").search(line):
            metatable[i]["rseqc_intron"] = int(line.split()[2])
            metatable[i]["intron_rpk"] = float(line.split()[-1])

    metatable[i]["exint_ratio"] =  metatable[i]["cds_rpk"]/metatable[i]["intron_rpk"]

#### Pull preseq data

In [19]:
for i in range(len(metatable)):
    paper_id = metatable[i][config["expt keys"]["paper_id"]]
    sample = metatable[i]["sample_name"]  
    dirpath = data_path + paper_id + "/qc/preseq/"
    filepath = dirpath + sample + ".lc_extrap.txt"
    
    if not (
        os.path.exists(filepath) and os.path.isfile(filepath)
    ):
        continue
           
    fdata = open(filepath)
    for line in fdata:
        if line.startswith("10000000.0"):
            distinct = float(line.split()[1])
            
    metatable[i]["distinct_tenmillion_prop"] = distinct/10000000

#### Pull pileup data

In [20]:
for i in range(len(metatable)):
    paper_id = metatable[i][config["expt keys"]["paper_id"]]
    sample = metatable[i]["sample_name"]
    dirpath = data_path + paper_id + "/qc/pileup/"
    filepath = dirpath + sample + ".coverage.stats.txt"
    
    if not (
        os.path.exists(filepath) and os.path.isfile(filepath)
    ):
        continue
         
    fdata = open(filepath)
    x = 0
    total = cov = fold = 0
    for line in fdata:
        if x == 0:
            x = x + 1
            continue
        else:
            x = x + 1
            total = total + int(line.split("\t")[2])
            cov = cov + int(line.split("\t")[5])
            fold = fold + float(line.split("\t")[1])*int(line.split("\t")[2])
        
    metatable[i]["genome_prop_cov"] = cov/total
    metatable[i]["avg_fold_cov"] = fold/total

#### Input data into database

In [21]:
accum_keys = list(dict(config["accum keys"]).values())

#with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url = "sqlite:///" + db_url) as session:
    for i in range(len(metatable)):
        entry = sampleAccum(
            sample_id = metatable[i]["sample_id"],
            replicate = metatable[i][config["accum keys"]["replicate"]],
            single_paired = metatable[i][config["accum keys"]["single_paired"]],
            rcomp = metatable[i][config["accum keys"]["rcomp"]],
            raw_read_depth = metatable[i]["raw_read_depth"],
            trim_read_depth = metatable[i]["trim_read_depth"],
            raw_read_length = metatable[i]["raw_read_length"],
            duplication_picard = metatable[i]["duplication_picard"],
            single_map = metatable[i]["single_map"],
            multi_map = metatable[i]["multi_map"],
            map_prop = metatable[i]["map_prop"],
            rseqc_tags = metatable[i]["rseqc_tags"],
            rseqc_cds = metatable[i]["rseqc_cds"],
            rseqc_five_utr = metatable[i]["rseqc_five_utr"],
            rseqc_three_utr = metatable[i]["rseqc_three_utr"],
            rseqc_intron = metatable[i]["rseqc_intron"],
            cds_rpk = metatable[i]["cds_rpk"],
            intron_rpk = metatable[i]["intron_rpk"],
            exint_ratio = metatable[i]["exint_ratio"],
            distinct_tenmillion_prop = metatable[i]["distinct_tenmillion_prop"],
            genome_prop_cov = metatable[i]["genome_prop_cov"],
            avg_fold_cov = metatable[i]["avg_fold_cov"],
        )
        session.merge(entry)